### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [6]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [9]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Talk Time/Data/\
1-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s


In [10]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 88, 157, 0, 4, 1, 36, 10, 0, 60]"
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[55, 0, 28, 0, 64, 31]"
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[20, 67, 79, 5, 35, 0, 24]"
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[53, 35, 57, 40, 12, 3]"
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 2, 5, 54, 12, 3]"
...,...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[16, 30, 111, 10, 56, 3]"
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[54, 63, 11, 56, 34, 3]"
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s,"[8, 10, 5, 19, 148, 15]"
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s,"[86, 16, 69, 8, 10, 58, 69]"


In [11]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [12]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 88, 157, 0, 4, 1, 36, 10, 0, 60]"
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[55, 0, 28, 0, 64, 31]"
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[20, 67, 79, 5, 35, 0, 24]"
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[53, 35, 57, 40, 12, 3]"
4,dus weet je wat het is dit is de,73,76,dus weet je wat het is dit is de honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=73s,"[88, 36, 1, 10, 2, 5, 30, 5, 3]"
...,...,...,...,...,...,...,...
22171,iedereen die weet als je een,579,581,afhankelijk is want iedereen die weet als je e...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[172, 20, 36, 25, 1, 6]"
22172,en we weten dat we een,606,608,als we in het bedrijf werken en we weten dat w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...,"[8, 11, 117, 4, 11, 6]"
22173,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s,"[8, 10, 5, 19, 148, 15]"
22174,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s,"[86, 16, 69, 8, 10, 58, 69]"


In [13]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [14]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [15]:
#word_num_index_dict

In [16]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [17]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[128,
 130,
 135,
 136,
 137,
 138,
 139,
 140,
 144,
 147,
 149,
 151,
 153,
 26,
 155,
 156,
 159,
 163,
 166,
 168,
 169,
 170,
 176,
 177,
 178,
 180,
 185,
 190,
 191,
 194,
 195,
 196,
 199,
 74,
 76,
 78,
 82,
 87,
 93,
 94,
 98,
 100,
 101,
 107,
 108,
 109,
 110,
 126,
 127]

In [18]:
#Counter(index_list)

In [19]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 88, 157, 0, 4, 1, 36, 10, 0, 60]"
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[55, 0, 28, 0, 64, 31]"
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[20, 67, 79, 5, 35, 0, 24]"
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[53, 35, 57, 40, 12, 3]"
4,dus weet je wat het is dit is de,73,76,dus weet je wat het is dit is de honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=73s,"[88, 36, 1, 10, 2, 5, 30, 5, 3]"
...,...,...,...,...,...,...,...
82,hij kan leven kijk naar mij,264,267,jou oh alsjeblieft hij kan leven kijk naar mij...,8LeiQzS8YtI,https://www.youtube.com/watch?v=8LeiQzS8YtI&t=...,"[14, 39, 122, 134, 34, 61]"
83,weten waar jij bent met wie,213,216,weten waar jij bent met wie wanneer hoelang wa...,6QhQ4GVUhmI,https://www.youtube.com/watch?v=6QhQ4GVUhmI&t=...,"[117, 55, 38, 62, 22, 86]"
84,net anders deze deze deze deze deze,544,549,dit de volgorde net anders deze deze deze deze...,fFvRj3XYH_Y,https://www.youtube.com/watch?v=fFvRj3XYH_Y&t=...,"[129, 167, 81, 81, 81, 81, 81]"
85,van jaar na jaar na jaar,204,206,we kijken van maand na maand en ook als we kij...,GOkG8Yq9ANo,https://www.youtube.com/watch?v=GOkG8Yq9ANo&t=...,"[9, 150, 192, 150, 192, 150]"


In [20]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[8, 88, 157, 0, 4, 1, 36, 10, 0, 60]"
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[55, 0, 28, 0, 64, 31]"
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[20, 67, 79, 5, 35, 0, 24]"
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...,"[53, 35, 57, 40, 12, 3]"
4,dus weet je wat het is dit is de,73,76,dus weet je wat het is dit is de honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=73s,"[88, 36, 1, 10, 2, 5, 30, 5, 3]"
...,...,...,...,...,...,...,...
82,hij kan leven kijk naar mij,264,267,jou oh alsjeblieft hij kan leven kijk naar mij...,8LeiQzS8YtI,https://www.youtube.com/watch?v=8LeiQzS8YtI&t=...,"[14, 39, 122, 134, 34, 61]"
83,weten waar jij bent met wie,213,216,weten waar jij bent met wie wanneer hoelang wa...,6QhQ4GVUhmI,https://www.youtube.com/watch?v=6QhQ4GVUhmI&t=...,"[117, 55, 38, 62, 22, 86]"
84,net anders deze deze deze deze deze,544,549,dit de volgorde net anders deze deze deze deze...,fFvRj3XYH_Y,https://www.youtube.com/watch?v=fFvRj3XYH_Y&t=...,"[129, 167, 81, 81, 81, 81, 81]"
85,van jaar na jaar na jaar,204,206,we kijken van maand na maand en ook als we kij...,GOkG8Yq9ANo,https://www.youtube.com/watch?v=GOkG8Yq9ANo&t=...,"[9, 150, 192, 150, 192, 150]"


In [21]:
df_youtube_sent_sample.drop("word_index", axis=1, inplace=True)

In [22]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx", index=False)

##### Selected Result Word Count

In [23]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
4,dus weet je wat het is dit is de,73,76,dus weet je wat het is dit is de honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=73s
...,...,...,...,...,...,...
82,hij kan leven kijk naar mij,264,267,jou oh alsjeblieft hij kan leven kijk naar mij...,8LeiQzS8YtI,https://www.youtube.com/watch?v=8LeiQzS8YtI&t=...
83,weten waar jij bent met wie,213,216,weten waar jij bent met wie wanneer hoelang wa...,6QhQ4GVUhmI,https://www.youtube.com/watch?v=6QhQ4GVUhmI&t=...
84,net anders deze deze deze deze deze,544,549,dit de volgorde net anders deze deze deze deze...,fFvRj3XYH_Y,https://www.youtube.com/watch?v=fFvRj3XYH_Y&t=...
85,van jaar na jaar na jaar,204,206,we kijken van maand na maand en ook als we kij...,GOkG8Yq9ANo,https://www.youtube.com/watch?v=GOkG8Yq9ANo&t=...


In [24]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [25]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,aan,7
1,mee,7
2,goed,7
3,hebben,7
4,heel,7
...,...,...
146,onze,1
147,over,1
148,terug,1
149,toen,1


In [26]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [27]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [28]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [29]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [30]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['hou',
 'zeg',
 'wordt',
 'uw',
 'dood',
 'omdat',
 'z',
 'h',
 'altijd',
 'jouw',
 'hun',
 'vader',
 'huis',
 'n',
 'wacht',
 'zie',
 'wilde',
 'vrouw',
 'dank',
 'willen',
 'mr',
 'bedankt',
 'spijt',
 'niemand',
 'geld',
 'praten',
 'gezien',
 'zullen',
 'werd',
 'vind',
 'genoeg',
 'wist',
 'neem',
 'hebt',
 'zal',
 'had',
 'laat',
 'doe',
 'alleen',
 's',
 'ok',
 'man',
 'nooit',
 'zei',
 'iemand',
 'niets',
 'komt',
 'zeggen',
 'tegen']

In [31]:
set_not_in_sent_word = set(not_in_sent_word)

In [32]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [33]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'alleen',
 'altijd',
 'bedankt',
 'dank',
 'doe',
 'dood',
 'geld',
 'genoeg',
 'gezien',
 'h',
 'had',
 'hebt',
 'hou',
 'huis',
 'hun',
 'iemand',
 'jouw',
 'komt',
 'laat',
 'man',
 'mr',
 'n',
 'neem',
 'niemand',
 'niets',
 'nooit',
 'ok',
 'omdat',
 'praten',
 's',
 'spijt',
 'tegen',
 'uw',
 'vader',
 'vind',
 'vrouw',
 'wacht',
 'werd',
 'wilde',
 'willen',
 'wist',
 'wordt',
 'z',
 'zal',
 'zeg',
 'zeggen',
 'zei',
 'zie',
 'zullen'}

In [34]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['mr',
 'vind',
 'omdat',
 'niets',
 'ok',
 'doe',
 'hun',
 'altijd',
 'man',
 'nooit',
 'spijt',
 'had',
 'uw',
 'wacht',
 'wist',
 'zeggen',
 'willen',
 'tegen',
 'dood',
 'geld',
 's',
 'iemand',
 'laat',
 'komt',
 'zie',
 'huis',
 'jouw',
 'n',
 'zullen',
 'zei',
 'hebt',
 'z',
 'praten',
 'hou',
 'neem',
 'wordt',
 'bedankt',
 'zal',
 'werd',
 'genoeg',
 'h',
 'dank',
 'zeg',
 'wilde',
 'alleen',
 'vrouw',
 'niemand',
 'vader',
 'gezien']

In [35]:
len(not_in_sent_selected_word_list)

49

In [36]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
4,dus weet je wat het is dit is de,73,76,dus weet je wat het is dit is de honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=73s
...,...,...,...,...,...,...
22171,iedereen die weet als je een,579,581,afhankelijk is want iedereen die weet als je e...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
22172,en we weten dat we een,606,608,als we in het bedrijf werken en we weten dat w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
22173,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s
22174,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s


In [37]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [38]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [39]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,vind,gaan doen ik vind het heel,643,645,laten we iets leuks gaan doen ik vind het heel...,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...
1,vind,en wat vind je van ze ze zijn,163,166,en wat vind je van ze ze zijn geweldig erg goe...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...
2,vind,vind waar ik kan zijn nou dan,217,220,en een leuke haven vind waar ik kan zijn nou d...,1g9tiRKq2dY,https://www.youtube.com/watch?v=1g9tiRKq2dY&t=...
3,vind,nou wat vind je van de,172,174,nou wat vind je van de tocht tot nu toe vandaag,D7TUFyFgo8Y,https://www.youtube.com/watch?v=D7TUFyFgo8Y&t=...
4,vind,ik vind het zeker niet erg,250,252,ik vind het zeker niet erg wanneer klm voor be...,J4cxyBbqFNo,https://www.youtube.com/watch?v=J4cxyBbqFNo&t=...
...,...,...,...,...,...,...,...
227,gezien,ze worden nu nog gezien als de,51,53,ze worden nu nog gezien als de grondleggers va...,s1Iy1dpIyIA,https://www.youtube.com/watch?v=s1Iy1dpIyIA&t=51s
228,gezien,dus als je die nog niet gezien hebt,423,425,over die andere reis heb ik een vlog gemaakt d...,aH0J3mbYpqQ,https://www.youtube.com/watch?v=aH0J3mbYpqQ&t=...
229,gezien,die hebben jullie nog nooit gezien in mijn,28,31,ik ga samen met floor die hebben jullie nog no...,1FCvmu0tdBw,https://www.youtube.com/watch?v=1FCvmu0tdBw&t=28s
230,gezien,als je hem niet hebt gezien kan je hem,19,21,als je hem niet hebt gezien kan je hem hierbov...,gfuVcOeD4EA,https://www.youtube.com/watch?v=gfuVcOeD4EA&t=19s


In [40]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [41]:
not_in_sent_selected_word_list_result.word.nunique()

47

In [42]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [43]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

{'mr', 'z'}

In [44]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [45]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,vind,gaan doen ik vind het heel,643,645,laten we iets leuks gaan doen ik vind het heel...,UltPsmBVTV0,https://www.youtube.com/watch?v=UltPsmBVTV0&t=...
1,vind,en wat vind je van ze ze zijn,163,166,en wat vind je van ze ze zijn geweldig erg goe...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...
2,vind,vind waar ik kan zijn nou dan,217,220,en een leuke haven vind waar ik kan zijn nou d...,1g9tiRKq2dY,https://www.youtube.com/watch?v=1g9tiRKq2dY&t=...
3,vind,nou wat vind je van de,172,174,nou wat vind je van de tocht tot nu toe vandaag,D7TUFyFgo8Y,https://www.youtube.com/watch?v=D7TUFyFgo8Y&t=...
4,vind,ik vind het zeker niet erg,250,252,ik vind het zeker niet erg wanneer klm voor be...,J4cxyBbqFNo,https://www.youtube.com/watch?v=J4cxyBbqFNo&t=...
...,...,...,...,...,...,...,...
227,gezien,ze worden nu nog gezien als de,51,53,ze worden nu nog gezien als de grondleggers va...,s1Iy1dpIyIA,https://www.youtube.com/watch?v=s1Iy1dpIyIA&t=51s
228,gezien,dus als je die nog niet gezien hebt,423,425,over die andere reis heb ik een vlog gemaakt d...,aH0J3mbYpqQ,https://www.youtube.com/watch?v=aH0J3mbYpqQ&t=...
229,gezien,die hebben jullie nog nooit gezien in mijn,28,31,ik ga samen met floor die hebben jullie nog no...,1FCvmu0tdBw,https://www.youtube.com/watch?v=1FCvmu0tdBw&t=28s
230,gezien,als je hem niet hebt gezien kan je hem,19,21,als je hem niet hebt gezien kan je hem hierbov...,gfuVcOeD4EA,https://www.youtube.com/watch?v=gfuVcOeD4EA&t=19s


In [46]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [47]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Sample2.xlsx']

In [48]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [49]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass